<a href="https://colab.research.google.com/github/ngonam2403/VEF-Final-Project_202007/blob/master/Chu%E1%BA%A9n_ho%C3%A1_c%C3%A1c_h%C3%A0m_th%C3%B4ng_d%E1%BB%A5ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mục đích: để tra cứu & tái sử dụng
---
Format chung: 
1. mục đích hàm
2. input (dataframe, list) --> output (number, dataframe, chart)
3. cần library nào để chạy? pandas, numpy, scikitlearn, seaborn

---
Cách đặt tên phần tử (Naming Convention): mục đích của việc này là để người khác dễ đọc và mường tượng được code của mình.
thứ tự ưu tiên: mục đích -> loại -> tên riêng để định danh.
- hàm: động từ, ví dụ def create_table() hoặc def plot_chart().
- biến: danh từ
  - dataframe: df_, ví dụ: thay vì viết data = pd.read_csv() thì viết df_abc = pd.read_csv(). nếu dataframe là một param trong hàm thì nên viết def create_table(param_df_xyz).
  - một con số: num_
  - text: str_
    
- lúc dùng thì ghi rõ param1 = , param2 =, param3 = 
- khi hàm có quá nhiều param thì chuyển sang dùng class,
- class: CamelCase

- tham khảo: https://www.analyticsvidhya.com/blog/2020/07/python-style-guide/; https://towardsdatascience.com/data-scientists-your-variable-names-are-awful-heres-how-to-fix-them-89053d2855be
---
flow chạy, pipeline.


In [ ]:
# Don't do this
temp = get_house_price_in_usd(house_sqft, house_room_count)
final_value = temp * usd_to_aud_conversion_rate
# Do this instead
house_price_in_usd = get_house_price_in_usd(house_sqft, 
                                            house_room_count)
house_price_in_aud = house_price_in_usd * usd_to_aud_conversion_rate

In [ ]:
for row_index in range(row_count):
    for column_index in range(column_count):
        ....

The benefits of adopting standards are that they let you make a single global decision instead of many local ones.

#0.Basic Statistics

In [ ]:
# app_train = pd.read_csv('../input/application_train.csv')

print('Training data shape: ', app_train.shape) #df.shape

app_train.info() #1 #better than df.describe()
app_train.dtypes #2
app_train.columns
app_train.describe() #3


#column type
# Number of each type of column
app_train.dtypes.value_counts()



# 1.Các hàm biến đổi-clean-transform dữ liệu



##1.1. Xử lý missing data 

In [ ]:
"""
FUNCTION: CHECK MISSING DATA
input: a dataframe MxN --> output: a dataframe Nx2
need: pandas
"""
def create_table_missing_values(df):
    mis_val = df.isnull().sum() #total missing values #input: df MxN -> output: df Nx2
    mis_val_percent = 100 * df.isnull().sum() / len(df) #percentage of missing values #input: df MxN -> output: df Nx2
    mis_val_table = pd.concat( #join df mis_val and df mis_val_percent
        [mis_val, mis_val_percent], 
        axis=1
        ) 
    mis_val_table_ren_columns = mis_val_table.rename( #rename the columns
        columns = {
            0 : 'Missing values', 
            1 : '% of Total Values'
            }
        )
    mis_val_table_ren_columns = mis_val_table_ren_columns[ #sort the table by percentage of missing descending
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1) #exclude those has 0% missing.
    
    print("Your selected dataframe has ")    
    return mis_val_table_ren_columns


"""
FUNCTION: REMOVE COLUMNS THAT HAS MISSING DATA ABOVE A THRESOLD, default 90%.
input: a dataframe MxN --> output: a dataframe MxK
need: pandas
"""
def remove_column_missing(df, threshold = 90):
  tb_missing_column = create_table_missing_values(df)
  list_missing_column = list(tb_missing_column.index[tb_missing_column['% of Total Values'] > threshold ])
  df_removed = df.drop(columns = list_missing_column)
  return df_removed
  print('There are {} columns removed.'.format(len(list_missing_column)))

# missing_train = create_table_missing_values(app_train)
# missing_train.head(10)

# list_train_variables_missing = list(missing_train.index[missing_train['% of Total Values'] > 90]) #pd.index #list(pd.index)
# len(missing_train_vars) #0

# list_test_variables_missing = list(missing_test.index[missing_test['% of Total Values'] > 90])
# len(missing_test_vars) #0

# missing_columns = list(set(missing_test_vars + missing_train_vars))
# print('There are %d columns with more than 90%% missing in either the training or testing data.' % len(missing_columns))

# # Drop the missing columns
# train = train.drop(columns = missing_columns)
# test = test.drop(columns = missing_columns)

In [ ]:
"""FUNCTION: impute numerical"""
from sklearn.preprocessing import Imputer
application[numerical_list] = Imputer(strategy='median').fit_transform(
    application[numerical_list])

##1.2. Xử lý biến dạng categorical

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def encode_one_hot(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
def encode_label(app_train):
  # Create a label encoder object
  le = LabelEncoder()
  le_count = 0

  # Iterate through the columns
  for col in app_train:
      if app_train[col].dtype == 'object':
          # If 2 or fewer unique categories
          if len(list(app_train[col].unique())) <= 2:
              # Train on the training data
              le.fit(app_train[col])
              # Transform both training and testing data
              app_train[col] = le.transform(app_train[col])              
              # Keep track of how many columns were label encoded
              le_count += 1
              
  print('%d columns were label encoded.' % le_count)
  return app_train

# 2.Các hàm vẽ biểu đồ

## a. KDE

In [ ]:
"""FUNCTION: Kernel Density Estimation plot
input: a dataframe --> output: a plot
need: pandas, seaborn, matplotlib
"""
def create_plot_kdeplot(df, target_col_name, x_variable):

    plt.figure(figsize = (10, 8))

    # # KDE plot of loans that were repaid on time
    # sns.kdeplot(
    #     app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, 
    #     label = 'target == 0')

    # # KDE plot of loans which were not repaid on time
    # sns.kdeplot(
    #     app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, 
    #     label = 'target == 1')

    for i in target_values:
        sns.kdeplot(
            df.loc[df[target_col_name]== i, x_variable],
            label = 'target == {}'.format(i)
            )

    # Labeling of plot
    plt.xlabel('Age (years)'); 
    plt.ylabel('Density'); 
    plt.title('Distribution of Ages')
  

## b. Phân phối

In [ ]:
"""FUNCTION: plot stats https://www.kaggle.com/gpreda/home-credit-default-risk-extensive-eda
input: dataframe --> output: a plot
need pandas, matplotlib, seaborn.
"""
# def f_plot_stats(feature, label_rotation=False, horizontal_layout=True):


""" FUNCTION: plot distribution """
def create_plot_one_distribution(feature, color):
    plt.figure(figsize=(10,6)) #create a figure
    plt.title("Distribution of %s" % feature) #create a title for chart.
    sns.distplot( #
        application_train[feature].dropna(),
        color=color, 
        kde=True,
        bins=100)
    plt.show()   


""" FUNCTION: plot multiple features distribution"""
def create_plot_multiple_distribution_comp(column_list,nrow=2):
    
    i = 0
    t1 = application_train.loc[application_train['TARGET'] != 0]
    t0 = application_train.loc[application_train['TARGET'] == 0]

    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(nrow,2,figsize=(12,6*nrow))

    for feature in var:
        i += 1
        plt.subplot(nrow,2,i)
        sns.kdeplot(t1[feature], bw=0.5,label="TARGET = 1")
        sns.kdeplot(t0[feature], bw=0.5,label="TARGET = 0")
        plt.ylabel('Density plot', fontsize=12)
        plt.xlabel(feature, fontsize=12)
        locs, labels = plt.xticks()
        plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show()

## c. Histogram

In [ ]:
"""FUNCTION: plot histogram"""
def create_plot_histogram(df,col_name,titleName,xlabelName):
    df[col_name].plot.hist(title = titleName)
    plt.xlabel(xlabelName)
"""example
plot_histogram(df = data,
               col_name = 'DAYS_EMPLOYED',
               titleName = 'Days Employment Histogram',
               xlabelName = 'Days Employment')
"""               

# 3.Các hàm về correlation

In [ ]:
"""FUNTION: top 3 correlation with Target and bottom 3 correlation
input: a dataframe MxN --> output: a dataframe Nx2
need: pandas
"""
def create_table_top_corr(df, target_col_name):
    correlations = df.corr()[target_col_name].sort_values()

    print('Most Positive Correlations:\n', correlations.tail(3))
    print('\nMost Negative Correlations:\n', correlations.head(3))


# Calculate all correlations in dataframe
corrs = train.corr()

corrs = corrs.sort_values('TARGET', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))
# Ten most negative correlations
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

# 4.Logistic Regression

## with Sklearn Library

In [ ]:
# from sklearn.linear_model import LogisticRegression
# log_reg = LogisticRegression() 
# """
# create a model.
# type(log_reg)
# <class 'sklearn.linear_model._logistic.LogisticRegression'>
# """

# # Load data
# y = app_train['TARGET']
# X = app_train.drop(columns = ['TARGET'])
# # Cleansing & Transformation pipeline

# # Split X and y into training and testing sets
# from sklearn.cross_validation import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(
#                                           X,
#                                           y,
#                                           test_size=0.25,
#                                           random_state=0)


# # Fit
# log_reg.fit(X, y)
# """
# 'fit' is a method of class 'linear_model'. it receives 
# dataframe X and vector y, as input, 
# and return self. 
# after we run .fit, we can run .predict
# """

# # Predict
# y_pred = log_reg.predict(X_test) #log_reg.predict([[1.7], [1.5]])
# """
# 'predict' is a method.
# it receive dataframe X as input, 
# and return 'Predicted class label per sample'.
# """

# # Evaluation
# # 1.Confustion Matrix
# from sklearn import metrics
# df_confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
# df_confusion_matrix

# # 2.Accuracy-Precision-Recall, on test set
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred))
# print("Recall:",metrics.recall_score(y_test, y_pred))

# # 3.F1-score
# from sklearn.metrics import f1_score
# f1_score(
#       y_test, 
#       y_pred, 
#       average='weighted')
# --0.9731332318832974


In [ ]:
#Example
>>> from sklearn import datasets
>>> iris = datasets.load_iris()
>>> type(iris)
# <class 'sklearn.utils.Bunch'>
>>> X = iris["data"][:,3:]
>>> type(X)
# <class 'numpy.ndarray'>
>>> y = (iris["target"]==2).astype(np.int)
>>> type(y)
# <class 'numpy.ndarray'>
>>> from sklearn.linear_model import LogisticRegression
>>> LogisticRegression().fit(X,y)
# LogisticRegression()

>>> X.shape
(150, 1)
>>> y.shape
(150,)
>>> X_train,X_test,y_train,y_test=train_test_split(
...                                           X,
...                                           y,
...                                           test_size=0.25,
...                                           random_state=0)
>>> LogisticRegression().fit(X_train,y_train)
# LogisticRegression()
>>> y_pred = log_reg.predict(X_test)
>>> from sklearn import metrics
>>> df_confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
>>> df_confusion_matrix
# array([[29,  0],
#        [ 1,  8]])

### why need .predict_proba() ???

In [ ]:
# https://stats.stackexchange.com/questions/391681/logisticregression-sklearn-why-does-predict-proba-yield-better-results/391757

# check classification scores of logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
# print('Train/Test split results:')
# print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
# print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

### cross validation with Logistic Regression

In [ ]:
# 10-fold cross-validation logistic regression
logreg = LogisticRegression()
# Use cross_val_score function
# We are passing the entirety of X and y, not X_train or y_train, it takes care of splitting the data
# cv=10 for 10 folds
# scoring = {'accuracy', 'neg_log_loss', 'roc_auc'} for evaluation metric - althought they are many
scores_accuracy = cross_val_score(logreg, X, y, cv=10, scoring='accuracy')
# scores_log_loss = cross_val_score(logreg, X, y, cv=10, scoring='neg_log_loss')
# scores_auc = cross_val_score(logreg, X, y, cv=10, scoring='roc_auc')
print('K-fold cross-validation results:')
print(logreg.__class__.__name__+" average accuracy is %2.3f" % scores_accuracy.mean())
# print(logreg.__class__.__name__+" average log_loss is %2.3f" % -scores_log_loss.mean())
# print(logreg.__class__.__name__+" average auc is %2.3f" % scores_auc.mean())

## logistic regression, with Statsmodel library

In [ ]:
import statsmodels.api as sm
# excl = ['mean radius','mean perimeter','mean area',
#         'worst area','worst perimeter','worst radius',
#         'radius error','perimeter error','area error']
# X_train = scaler.fit_transform(X_train.drop(excl,axis=1))


model = sm.Logit(y_train, X_train).fit()
print(model.summary())

y_scores = model.predict(scaler.fit_transform(X_test.drop(excl,axis=1)))

y_pred = (y_scores >0.5).astype(int)

accuracy_score(y_test.values.ravel(), y_pred)
# 0.9210526315789473

# 5.Evaluation
 

## cross validation.

In [ ]:
#Example: with a classifier call SGDClassifier, from "Hands on ML/Chapter3"
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42) sgd_clf.fit(X_train, y_train_5)

#Cross Validation
from sklearn.model_selection import cross_val_score
#Cross Validation with "Accuracy"
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy") 
# array([0.96355, 0.93795, 0.95615])


from sklearn.model_selection import cross_val_predict 
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred) 
#array([[53057, 1522], [ 1325, 4096]])


from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5, y_train_pred) # == 4096 / (4096 + 1522)
#0.7290850836596654
recall_score(y_train_5, y_train_pred) # == 4096 / (4096 + 1325)
#0.7555801512636044
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)
#0.7420962043663375

In [ ]:
#Another example with a classifier call RandomForestClassifier 
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42) 
y_probas_forest = cross_val_predict(
                            forest_clf, 
                            X_train, 
                            y_train_5, 
                            cv=3, 
                            method="predict_proba")

Some algorithms (such as SGD classifiers, Random Forest classifiers, and naive Bayes classifiers) are capable of handling multiple classes natively. Others (such as Logistic Regression or Support Vector Machine classifiers) are strictly binary classifiers.

In [ ]:
# #multiclass classifier Confusion Matrix looks like this
# >>> y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)

# >>> conf_mx = confusion_matrix(y_train, y_train_pred)

# >>> conf_mx 
# array([[5578, 0, 22, 7, 8, 45, 35, 5, 222, 1], 
#        [ 0, 6410, 35, 26, 4, 44, 4, 8, 198, 13], 
#        [ 28, 27, 5232, 100, 74, 27, 68, 37, 354, 11], 
#        [ 23, 18, 115, 5254, 2, 209, 26, 38, 373, 73], 
#        [ 11, 14, 45, 12, 5219, 11, 33, 26, 299, 172], 
#        [ 26, 16, 31, 173, 54, 4484, 76, 14, 482, 65], 
#        [ 31, 17, 45, 2, 42, 98, 5556, 3, 123, 1], 
#        [ 20, 10, 53, 27, 50, 13, 3, 5696, 173, 220], 
#        [ 17, 64, 47, 91, 3, 125, 24, 11, 5421, 48], 
#        [ 24, 18, 29, 67, 116, 39, 1, 174, 329, 5152]])

In [ ]:
#An example of SVM Classifier 
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import LinearSVC

iris = datasets.load_iris() 
X = iris["data"][:, (2, 3)] # petal length, petal width 
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([ 
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1, loss="hinge")),
                    ])

svm_clf.fit(X, y)

>>> svm_clf.predict([[5.5, 1.7]])
# array([1.])

## confusion matrix 

## F1-score

In [ ]:
from sklearn.metrics import 
score = sklearn.metrics.f1_score(y_test, y_pred)

## ROC AUC score

An AUC score of 0.5 is effectively as good as the flip of a coin, and means that the model really has no classification power at all between the positive and negative occurences. 

"AUC of a classifier is equivalent to the probability that the classifier will rank a randomly chosen positive instance higher than a randomly chosen negative instance." -Majnik, Bosnic, 2011

https://www.kaggle.com/sgus1318/titanic-analysis-learning-to-swim-with-python

In [ ]:
# (Nam điền code ROC AUC vào đây)

from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, y_pred)


# 6.Các hàm chưa phân loại

In [ ]:
# Chọn, select data từ một dataframe
""" select from where [condition] """

# Thay vì viết
df_abc = application_train[application_train.TARGET==1]
# Nên viết
df_abc = application_train.loc[application_train.TARGET==1]

In [ ]:
def model(train_features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = train_features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = train_features['TARGET']
    
    # Remove the ids and target
    train_features = train_features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        train_features = pd.get_dummies(train_features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        train_features, test_features = train_features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(train_features):
            if train_features[col].dtype == 'object':
                # Map the categorical features to integers
                train_features[col] = label_encoder.fit_transform(np.array(train_features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', train_features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(train_features.columns)
    
    # Convert to np arrays
    train_features = np.array(train_features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(train_features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(train_features):
        
        # Training data for the fold
        train_features, train_labels = train_features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = train_features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics


## Join data

### Aggregate variables from 6 other tables

### pandas MERGE & pandas JOIN

In [ ]:
""" 
pd.merge https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

-left vs inner join: df1.join(df2) does a left join by default (keeps all rows of df1), 
but df.merge does an inner join by default (returns only matching rows of df1 and df2).
-So, the generic approach is to use pandas.merge(df1, df2) or df1.merge(df2). 
But for a number of common situations (keeping all rows of df1 and joining 
to an index in df2), you can save some typing by using df1.join(df2) instead.
https://stackoverflow.com/questions/22676081/what-is-the-difference-between-join-and-merge-in-pandas

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
"""


train_labels = train_bureau['TARGET']
previous_features.append('SK_ID_CURR')

train_ids = train_bureau['SK_ID_CURR']
test_ids = test_bureau['SK_ID_CURR']

# Merge the dataframes avoiding duplicating columns by subsetting train_previous
train = train_bureau.merge(train_previous[previous_features], on = 'SK_ID_CURR') #pd.dataframe.merge()
test = test_bureau.merge(test_previous[previous_features], on = 'SK_ID_CURR')

In [ ]:
#align 
# Match the columns in the dataframes
train, test = train.align(test, join = 'inner', axis = 1)
print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

In [ ]:
def merge(df):
    df = df.join(data_bureau_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    df = df.join(bureau_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.join(data_credit_card_balance_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
    df = df.join(data_previous_application_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
    df = df.join(data_installments_payments_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    
    return df

train = merge(app_train)
test = merge(app_test)

# Feature Selection

## remove Collinear variables

In [ ]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = train.corr().abs()
corr_matrix.head()

# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))

train = train.drop(columns = to_drop)
test = test.drop(columns = to_drop)

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

## select top 15 features to keep

In [ ]:
# # Threshold for cumulative importance
# threshold = 0.95

# # Extract the features to keep
# features_to_keep = list(norm_feature_importances[norm_feature_importances['cumulative_importance'] < threshold]['feature'])

# # Create new datasets with smaller features
# train_small = train[features_to_keep]
# test_small = test[features_to_keep]

In [ ]:
# Cach 1
list_column = []
selected_X_train = X_train[list_column]

# Cach 2
features = list(train_gr.columns)
selected_X_train = X_train[features]

## Dimensionality Reduction

### PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 154) 
X_reduced = pca.fit_transform(X_train) 
X_recovered = pca.inverse_transform(X_reduced)

# HOAC 
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

# Submit to Kaggle competition (cant do it now)

In [ ]:
final_test['Survived'] = log_clf.predict(final_test[Selected_features])
final_test['PassengerId'] = test_df['PassengerId']

submission = final_test[['PassengerId','Survived']]

submission.to_csv("submission.csv", index=False)

submission.tail()

# Explanable ML